### Installiamo i due pacchetti
Da far eseguire solo la prima volta

In [2]:
!pip install msgpack
!pip install pyspark==2.3.0


### Importiamo le librerie

In [3]:
import os
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[8] pyspark-shell"
os.environ["JAVA_HOME"] = "C:\jdk1.8.0_171"
#os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-10.0.1"


#USATO PER RISOLVEREE PROBLEMA JAVA GATEWAY ..... (ho anche installato java e inserito manualmente java home tra le variabili ambiente)

In [5]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA, BisectingKMeans
from pyspark.sql.functions import monotonically_increasing_id
import re
import json
import pandas as pd
import matplotlib.pyplot as plt
from nltk.stem.snowball import SnowballStemmer
import nltk
from nltk.corpus import stopwords
import pyspark
import string
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.mllib.util import MLUtils
from pyspark.sql.types import *
from pyspark.ml.feature import CountVectorizer, CountVectorizerModel, Tokenizer, RegexTokenizer, StopWordsRemover
nltk.download("stopwords")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\paperspace\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Creiamo la sessione di spark e importiamo il dataset da un documento json

Il campo di interesse è testo_txt_it

In [6]:
spark = SparkSession.builder.appName('NLP_topicModel').getOrCreate()


In [7]:
rawdata = spark.read.json("ARTICOLITV2.json", multiLine=True)
rawdata = rawdata.fillna({'testo_txt_it': ''})
rawdata = rawdata.withColumn("uid", monotonically_increasing_id())
rawdata = rawdata.withColumn("year_month", rawdata.published_dt.substr(1,7))
rawdata.show(3)


+-------------------+--------------------+--------+----------+--------------------+--------------------+--------------------+---+----------+
|          _version_|                  id|language|language_s|        published_dt|        testo_txt_it|               url_s|uid|year_month|
+-------------------+--------------------+--------+----------+--------------------+--------------------+--------------------+---+----------+
|1600612814188183552|2cfcaaac52cfbb66d...|        |   italian|2018-05-15T14:08:00Z|| 0
Dal 1° al 10 ...|http://omgili.com...|  0|   2018-05|
|1600612814388461568|0cf575e54190bff2e...|        |   italian|2018-05-15T14:08:00Z|15 maggio 2018 16...|http://omgili.com...|  1|   2018-05|
|1600612814394753024|b8dbc4e7826b41575...|        |   italian|2018-05-15T14:08:00Z|Leonardo Nappi 13...|http://omgili.com...|  2|   2018-05|
+-------------------+--------------------+--------+----------+--------------------+--------------------+--------------------+---+----------+
only showing 

In [8]:
#def getrows(df, rownums=None):
#     return df.rdd.zipWithIndex().filter(lambda x: x[1] in rownums).map(lambda x: x[0])
#getrows(rawdata.select("testo_txt_it"), [2]).collect()
#rawdata.select("testo_txt_it").rdd.zipWithIndex().filter(lambda x: x[1] in [2]).map(lambda x: x[0])
#rawdata.select("testo_txt_it").show(3, False)

In [9]:
#sc = SparkContext.getOrCreate()
#sc._conf.getAll()

In [10]:
#rawdata.select("testo_txt_it").show(3, False)#

# Puliamo i dati:
- Rimuoviamo la punteggiatura
- Applichiamo la tokenization per avere singole parole
- Applichiamo lo stemming al nostro dataset
- Rimuoviamo le stopwords:
    - Rimuoviamo le parole che sono composte da meno di 3 lettere
    - Rimuoviamo le parole che capitano frequentemente nei nostri documenti
    - Rimuoviamo le stopwords del pacchetto NLTK (italiane e inglesi)
    - Rimuoviamo le stopwords di PySpark 
    - Rimuoviamo tutte queste stopwords applicandogli lo stemming

In [11]:
def myStemmer (record, array=False):
    stemmer = SnowballStemmer("italian")
    if array:
        text_out = [stemmer.stem(word) for word in record] 
    else:    
        text = record [2]#9
        text_out = [stemmer.stem(word) for word in text]
    return(text_out)

    

#def myCleaner (record):
#    text = record [10]
#    text_out = [re.sub("[^a-zA-Z]","", word.strip().lower()) for word in text]
#    text_out = [word for word in text if (word != "" and word != " ")]
#    return(text_out)

def removePunctuation(column):
    return trim(lower(regexp_replace(column, '[^A-Za-z,]', ' '))).alias('sentence')

In [12]:
# print("Contains digits",contains_digits)
# print("more_then_3_charachters",more_then_3_charachters)


In [13]:

prova = rawdata.select(removePunctuation(col("testo_txt_it")))
#Tokenize the text in the text column
tokenizer = Tokenizer(inputCol="sentence", outputCol="tokenized")#testo_txt_it
wordsDataFrame = tokenizer.transform(prova)#rawdata

remover = StopWordsRemover(inputCol="tokenized", outputCol="tokenizedNew", stopWords = [" ", ""])
wordsDataFrame = remover.transform(wordsDataFrame)

udf_myStemmer = udf(myStemmer, (ArrayType(StringType()))) # if the function returns an int
wordsDataFrame = wordsDataFrame.withColumn("stemmed", udf_myStemmer(struct([wordsDataFrame[x] for x in wordsDataFrame.columns])))

# udf_myFunction = udf(myCleaner, (ArrayType(StringType()))) # if the function returns an int
# wordsDataFrame = wordsDataFrame.withColumn("cleaned", udf_myFunction(struct([wordsDataFrame[x] for x in wordsDataFrame.columns])))

# #remove 20 most occuring documents, documents with non numeric characters, and documents with <= 3 characters
cv_tmp = CountVectorizer(inputCol="stemmed", outputCol="tmp_vectors")
cv_tmp_model = cv_tmp.fit(wordsDataFrame)

#wordsDataFrame.show(5)



top100 = list(cv_tmp_model.vocabulary[0:100])
stopWordsCustom = [" ","", "dal", "lung", "tant", "al","davan","avev","qualc", "qualcuno", "qualcosa", "avevano", "davanti", "aveva","e","avere", "fare","la","li", "lo", "gli", "essere", "solo", "per", "cosa", "ieri","disponibile", "anno", "detto", "quando","fatto", "sotto", "alcuna", "quali"]
#Add additional stopwords in th, is list

more_then_3_charachters = [word for word in cv_tmp_model.vocabulary if len(word) <= 3 ]
stopWords = list(set(stopwords.words('english')))+list(set(stopwords.words('italian')))
#Combine all the stopwords
stpw = StopWordsRemover.loadDefaultStopWords("italian") + stopWords+ stopWordsCustom +top100 + more_then_3_charachters 
stem_stopw = myStemmer(stpw, True) #stemming the stopwords

#Remove stopwords from the tokenized list
removerNew = StopWordsRemover(inputCol="stemmed", outputCol="final", stopWords =   stem_stopw+stpw) 
clean_text= removerNew.transform(wordsDataFrame)#dropping the stemmed stopwords from the stemmed word

In [14]:
wordsDataFrame.select("Stemmed").show(5) #parole stemmate

+--------------------+
|             Stemmed|
+--------------------+
|[dal, al, giugn, ...|
|[magg, condivisio...|
|[leonard, napp, m...|
|[twitter, exop, o...|
|[mir, debutt, av,...|
+--------------------+
only showing top 5 rows



In [15]:
# reviews = data.map(lambda x : x['Review Text']).filter(lambda x: x is not None)
# StopWords = stopwords.words("english")
# tokens = reviews                                                   \
#     .map( lambda document: document.strip().lower())               \
#     .map( lambda document: re.split(" ", document))          \
#     .map( lambda word: [x for x in word if x.isalpha()])           \
#     .map( lambda word: [x for x in word if len(x) > 3] )           \
#     .map( lambda word: [x for x in word if x not in StopWords])    \
#     .zipWithIndex()

clean_text.select("final").show(5, False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [16]:
# def stem(in_vec):
#     out_vec = []
#     for t in in_vec:
#         t_stem = stemmer.stem(t)
#         if len(t_stem) > 2:
#             out_vec.append(t_stem)       
#     return out_vec

# # Create user defined function for stemming with return type Array<String>
# from pyspark.sql.types import *
# stemmer_udf = udf(lambda x: stem(x), ArrayType(StringType()))
# # Create new df with vectors containing the stemmed tokens 
# wordsDataFrame = (
#     wordsDataFrame
#         .withColumn("vector_stemmed", stemmer_udf("filtered"))
#         .select("vector_stemmed")
#   )

# # Rename df and column for clarity
# production_df = wordsDataFrame.select(col("vector_stemmed").alias("unigrams"))

# # Display
# production_df.printSchema()
# production_df.show()


### Dividiamo il dataset in train e test

In [17]:
train, test = clean_text.select("final").randomSplit([0.75, 0.25])
train.show(5)
test.show(5)

+--------------------+
|               final|
+--------------------+
|                  []|
|[accant, sardegn,...|
|[acciaier, venete...|
|[adozion, distanz...|
|[aere, scompars, ...|
+--------------------+
only showing top 5 rows

+--------------------+
|               final|
+--------------------+
|[acqua, econom, b...|
|[agcom, estend, a...|
|[aggrav, bilanc, ...|
|[aics, casert, vi...|
|[alasd, macleod, ...|
+--------------------+
only showing top 5 rows



In [18]:
#train = train.na.drop(how="all")
#train = train.drop(train.take(1))
#train.show()

# Creiamo la TF-IDF matrix

- tf: data da CountVectorizer
- idf: data da IDF, che prende in input il dataset ricavato dal CountVectorizer

In [19]:
#clean_text.select("final").show(3)
#cv = CountVectorizer(inputCol="final", outputCol="rawFeatures", vocabSize = 4000, minDF = 5)
#cvmodel = cv.fit(clean_text)
#featurizedData = cvmodel.transform(clean_text)
#featurizedData.select("final","rawFeatures").show(5)
cv_train = CountVectorizer(inputCol="final", outputCol="rawFeatures", vocabSize = 4000, minDF = 5)
cvmodel = cv_train.fit(train)
cvDatasetTrain = cvmodel.transform(train)
cvDatasetTrain.select("final","rawFeatures").show(5)
cvDatasetTest = cvmodel.transform(test)

+--------------------+--------------------+
|               final|         rawFeatures|
+--------------------+--------------------+
|                  []|        (3322,[],[])|
|[accant, sardegn,...|(3322,[6,8,12,13,...|
|[acciaier, venete...|(3322,[0,2,4,8,22...|
|[adozion, distanz...|(3322,[0,1,2,3,4,...|
|[aere, scompars, ...|(3322,[2,5,11,12,...|
+--------------------+--------------------+
only showing top 5 rows



In [20]:
vocab = cvmodel.vocabulary
vocab_broadcast = spark.sparkContext.broadcast(vocab)
idf = IDF(inputCol="rawFeatures", outputCol="features")
#idfModel = idf.fit(featurizedData)
#rescaledData = idfModel.transform(featurizedData)
idfModel = idf.fit(cvDatasetTrain)
rescaledDataTrain = idfModel.transform(cvDatasetTrain)
rescaledDataTest = idfModel.transform(cvDatasetTest)


In [21]:
#rescaledData.select("features").show(2, False)
    

# Modello

In [22]:
# Num_Argomenti = [10, 20, 50, 100, 150] #GRID SEARCH per decidere il numero di topic k
# loglik=[]
# logperp=[]
# n_iter = {Num_Argomenti[0]: 20, Num_Argomenti[1]: 40, Num_Argomenti[2]: 100, Num_Argomenti[3]:180, Num_Argomenti[4]: 260}
# for i in Num_Argomenti:
#     print(i)
#   http://localhost:8888/notebooks/Desktop/Materiale%20didattico/LDA/MIO_ULTIMO.ipynb#  lda = LDA(k=i, seed=123, optimizer="online", featuresCol="features", optimizeDocConcentration= True, maxIter= n_iter[i])
#     ldamodel = lda.fit(rescaledData)
#     loglik.append(ldamodel.logLikelihood(rescaledData))
#     logperp.append(ldamodel.logPerplexity(rescaledData))
#ldanew = LDA(k=15, seed=123, optimizer="online",featuresCol="features", subsamplingRate = 0.2,maxIter = 20)# optimizeDocConcentration= True, subsamplingRate = 0.2, learningDecay = 0.5, 
#ldamodelNEW = ldanew.fit(rescaledDataTrain)
k = 15
lda = LDA(k=k, seed=123, optimizer="online",featuresCol="features", maxIter = 2000 )# optimizeDocConcentration= True, subsamplingRate = 0.2, learningDecay = 0.5, 
ldamodel = lda.fit(rescaledDataTrain)
ldatopics = ldamodel.describeTopics()
ldatopics.show(k)


+-----+--------------------+--------------------+
|topic|         termIndices|         termWeights|
+-----+--------------------+--------------------+
|    0|[1738, 374, 1207,...|[0.03053298034404...|
|    1|[912, 720, 1711, ...|[0.03944533340000...|
|    2|[313, 429, 891, 1...|[0.05238150389240...|
|    3|[171, 76, 277, 20...|[0.00773294918387...|
|    4|[261, 294, 253, 4...|[0.07005991460847...|
|    5|[967, 763, 418, 1...|[0.02425414231040...|
|    6|[318, 630, 159, 1...|[0.03368926436323...|
|    7|[943, 443, 197, 8...|[0.01112690965654...|
|    8|[82, 147, 122, 59...|[0.00774487301626...|
|    9|[327, 126, 121, 3...|[0.01216219696826...|
|   10|[1137, 1383, 1531...|[0.04610724646007...|
|   11|[280, 507, 22, 29...|[0.00796826725430...|
|   12|[239, 27, 190, 48...|[0.01692770559317...|
|   13|[284, 30, 454, 14...|[0.00804463792291...|
|   14|[12, 131, 163, 15...|[0.00539198014365...|
+-----+--------------------+--------------------+



In [23]:
prediction = ldamodel.transform(rescaledDataTest) #previsioni sul test

In [71]:
#prediction.select("topicDistribution").show(2, False)
local_model_path ="C:/Users/paperspace/Desktop/LDA/lda"
ldamodel.save(local_model_path)
#provando = LDA.load("/lda2000it")
#spark.sparkContext.getLocalProperty

Py4JJavaError: An error occurred while calling o1052.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:96)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:363)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$4.apply$mcV$sp(PairRDDFunctions.scala:1067)
	at org.apache.spark.rdd.PairRDDFunctions$$

In [69]:
import numpy as np
print("Train-set results")
print("LogLikelihood", np.round(ldamodel.logLikelihood(rescaledDataTrain),2), "LogPerplexity", np.round(ldamodel.logPerplexity(rescaledDataTrain),2))

Train-set results
LogLikelihood -1930441.43 LogPerplexity 7.44


In [70]:
# import numpy as np
# plt.plot(Num_Argomenti, loglik)
# print(logperp)
# print(loglik)
print("Test-set results")
print("LogLikelihood", np.round(ldamodel.logLikelihood(rescaledDataTest),2), "LogPerplexity", np.round(ldamodel.logPerplexity(rescaledDataTest),2))

Test-set results
LogLikelihood -718901.52 LogPerplexity 8.05


## Topic description

Per ogni topic abbiamo un insieme di parole che lo descrivono, io ho scelto di memorizzare le prime 10 e mostrarle a schermo in modo da poter capire di che topic si tratta

In [31]:

ldatopics = ldamodel.describeTopics()
ldatopics.show(150)
def map_termID_to_Word(termIndices):
    words = []
    for termID in termIndices:
        words.append(vocab_broadcast.value[termID])
    
    return words

udf_map_termID_to_Word = udf(map_termID_to_Word , ArrayType(StringType()))
ldatopics_mapped = ldatopics.withColumn("topic_desc", udf_map_termID_to_Word(ldatopics.termIndices))
ldaResults = ldamodel.transform(rescaledDataTrain)

ldatopics_mapped.select(ldatopics_mapped.topic, ldatopics_mapped.topic_desc).show(150,False)

+-----+--------------------+--------------------+
|topic|         termIndices|         termWeights|
+-----+--------------------+--------------------+
|    0|[1738, 374, 1207,...|[0.03053298034404...|
|    1|[912, 720, 1711, ...|[0.03944533340000...|
|    2|[313, 429, 891, 1...|[0.05238150389240...|
|    3|[171, 76, 277, 20...|[0.00773294918387...|
|    4|[261, 294, 253, 4...|[0.07005991460847...|
|    5|[967, 763, 418, 1...|[0.02425414231040...|
|    6|[318, 630, 159, 1...|[0.03368926436323...|
|    7|[943, 443, 197, 8...|[0.01112690965654...|
|    8|[82, 147, 122, 59...|[0.00774487301626...|
|    9|[327, 126, 121, 3...|[0.01216219696826...|
|   10|[1137, 1383, 1531...|[0.04610724646007...|
|   11|[280, 507, 22, 29...|[0.00796826725430...|
|   12|[239, 27, 190, 48...|[0.01692770559317...|
|   13|[284, 30, 454, 14...|[0.00804463792291...|
|   14|[12, 131, 163, 15...|[0.00539198014365...|
+-----+--------------------+--------------------+

+-----+------------------------------------------

In [35]:
def breakout_array(index_number, record):
    vectorlist = record.tolist()
    return vectorlist[index_number]

udf_breakout_array = udf(breakout_array, FloatType())

# In questo modo possiamo mostrare per ogni documento la percentuale di appartenenza ad un certo topics   


Da sistemare


In [37]:

# #enrichedData = ldaResults.withColumn("Topic_0", udf_breakout_array(lit(0), ldaResults.topicDistribution)).withColumn("Topic_1", udf_breakout_array(lit(1), ldaResults.topicDistribution)).withColumn("Topic_2", udf_breakout_array(lit(2), ldaResults.topicDistribution)).withColumn("Topic_3", udf_breakout_array(lit(3), ldaResults.topicDistribution)).withColumn("Topic_4", udf_breakout_array(lit(4), ldaResults.topicDistribution)).withColumn("Topic_5", udf_breakout_array(lit(5), ldaResults.topicDistribution)).withColumn("Topic_6", udf_breakout_array(lit(6), ldaResults.topicDistribution)).withColumn("Topic_7", udf_breakout_array(lit(7), ldaResults.topicDistribution)).withColumn("Topic_8", udf_breakout_array(lit(8), ldaResults.topicDistribution)).withColumn("Topic_9", udf_breakout_array(lit(9), ldaResults.topicDistribution)).withColumn("Topic_10", udf_breakout_array(lit(10), ldaResults.topicDistribution)).withColumn("Topic_11", udf_breakout_array(lit(11), ldaResults.topicDistribution)).withColumn("Topic_12", udf_breakout_array(lit(12), ldaResults.topicDistribution)).withColumn("Topic_13", udf_breakout_array(lit(13), ldaResults.topicDistribution)).withColumn("Topic_14", udf_breakout_array(lit(14), ldaResults.topicDistribution)).withColumn("Topic_15", udf_breakout_array(lit(15), ldaResults.topicDistribution))    

# enrichedData = prediction \
#         .withColumn("Topic_0", udf_breakout_array(lit(0), prediction.topicDistribution))    \
#         .withColumn("Topic_1", udf_breakout_array(lit(1), prediction.topicDistribution))    \
#         .withColumn("Topic_2", udf_breakout_array(lit(2), prediction.topicDistribution))    \
#         .withColumn("Topic_3", udf_breakout_array(lit(3), prediction.topicDistribution))    \
#         .withColumn("Topic_4", udf_breakout_array(lit(4), prediction.topicDistribution))    \
#         .withColumn("Topic_5", udf_breakout_array(lit(5), prediction.topicDistribution))    \
#         .withColumn("Topic_6", udf_breakout_array(lit(6), prediction.topicDistribution))    \
#         .withColumn("Topic_7", udf_breakout_array(lit(7), prediction.topicDistribution))    \
#         .withColumn("Topic_8", udf_breakout_array(lit(8), prediction.topicDistribution))    \
#         .withColumn("Topic_9", udf_breakout_array(lit(9), prediction.topicDistribution))    
#         .withColumn("Topic_10", udf_breakout_array(lit(10), prediction.topicDistribution))  \
#         .withColumn("Topic_11", udf_breakout_array(lit(11), prediction.topicDistribution))  \
#         .withColumn("Topic_12", udf_breakout_array(lit(12), prediction.topicDistribution))  \
#         .withColumn("Topic_13", udf_breakout_array(lit(13), prediction.topicDistribution))  \
#         .withColumn("Topic_14", udf_breakout_array(lit(14), prediction.topicDistribution))  

In [38]:
# enrichedData.show()

+--------------------+--------------------+--------------------+--------------------+-------------+------------+------------+------------+------------+
|               final|         rawFeatures|            features|   topicDistribution|     Topic_10|    Topic_11|    Topic_12|    Topic_13|    Topic_14|
+--------------------+--------------------+--------------------+--------------------+-------------+------------+------------+------------+------------+
|[acqua, econom, b...|(3322,[0,1,2,3,6,...|(3322,[0,1,2,3,6,...|[3.78855692362897...| 0.0054824986| 0.076581135|  0.02965965| 0.047862526| 0.062499925|
|[agcom, estend, a...|(3322,[0,1,12,13,...|(3322,[0,1,12,13,...|[7.82727893993767...|  0.017529093| 2.413132E-4|1.9457372E-4|  0.22001347|  0.09607001|
|[aggrav, bilanc, ...|(3322,[0,1,5,6,7,...|(3322,[0,1,5,6,7,...|[3.72983733691397...| 2.4193954E-5|    0.535461| 0.014609428| 0.100260206| 0.028937029|
|[aics, casert, vi...|(3322,[5,7,11,15,...|(3322,[5,7,11,15,...|[0.01964151795991...| 9.